#### **CSV RAG**

 - We'll load and split csv file into relevant text chunks 
 - Create a vectorstore / make a retriever
 - Apply LLM to give a response

#### **Set your LLM and embedding model**

In [3]:
# same from ollama we'll use llama3.2
from langchain_ollama import ChatOllama

# this is LLM we'll use
llm = ChatOllama(
    model="llama3.2",
    temperature=0,
    verbose=True
)

llm.invoke("How are you?")

AIMessage(content="I'm just a language model, so I don't have feelings or emotions like humans do. However, I'm functioning properly and ready to assist you with any questions or tasks you may have! How can I help you today?", additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-12-03T11:31:25.357692Z', 'done': True, 'done_reason': 'stop', 'total_duration': 21065541791, 'load_duration': 3630263750, 'prompt_eval_count': 29, 'prompt_eval_duration': 13063766250, 'eval_count': 47, 'eval_duration': 3049866871, 'logprobs': None, 'model_name': 'llama3.2', 'model_provider': 'ollama'}, id='lc_run--1c96b4a3-31a3-403e-810c-8e0776b52a0d-0', usage_metadata={'input_tokens': 29, 'output_tokens': 47, 'total_tokens': 76})

In [4]:
# for embedding model we'll use sentence-transformers
from langchain_huggingface import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')

# sample embedding 
embeddings = embedding_model.embed_query("Hey How are you?")
print(f"Length of embeddings : {len(embeddings)}")
print(f"Embedding : {embeddings[:100]}")

Length of embeddings : 384
Embedding : [-0.013380538672208786, 0.003255972173064947, 0.10806030035018921, 0.08322358131408691, 0.02040085941553116, -0.049066152423620224, 0.0722508355975151, 0.002980925841256976, -0.08823534101247787, 0.016058299690485, -0.03367079421877861, -4.332493062975118e-06, -0.02510129101574421, 0.0007887802203185856, 0.060331884771585464, -0.0415474958717823, 0.07702311128377914, -0.14256997406482697, -0.13958506286144257, 0.06023767963051796, 0.003192346775904298, 0.018982844427227974, 0.02300790697336197, 0.06056844815611839, -0.07911035418510437, -0.05399537831544876, -0.0008475205395370722, 0.03202424943447113, -0.029674910008907318, -0.04484577104449272, -0.10411098599433899, 0.06399180740118027, -0.05713418126106262, -0.02695028856396675, -0.028776653110980988, 0.00333896791562438, -0.0355900302529335, -0.13525626063346863, 0.009469274431467056, 0.0003555373114068061, 0.009924577549099922, -0.0014938903041183949, -0.009747199714183807, -0.002170604653656

## Step1 : Loading CSV -> Chunking -> Vectorstore

In [5]:
from langchain_community.document_loaders.csv_loader import CSVLoader

loader = CSVLoader("../data/customers-100.csv")
docs = loader.load_and_split()

print(f"Number of Docs : {len(docs)}")

Number of Docs : 100


In [6]:
from pprint import pprint
pprint(docs[1].page_content)

('Index: 2\n'
 'Customer Id: 1Ef7b82A4CAAD10\n'
 'First Name: Preston\n'
 'Last Name: Lozano\n'
 'Company: Vega-Gentry\n'
 'City: East Jimmychester\n'
 'Country: Djibouti\n'
 'Phone 1: 5153435776\n'
 'Phone 2: 686-620-1820x944\n'
 'Email: vmata@colon.com\n'
 'Subscription Date: 2021-04-23\n'
 'Website: http://www.hobbs.com/')


So you can see content is loaded row-wise

In [7]:
## create a vectorstore
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

index = faiss.IndexFlatL2(len(embedding_model.embed_query("hello world")))

vector_store = FAISS(
    embedding_function=embedding_model,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

## adding docs to vectorstore
vector_store.add_documents(documents=docs)

['f3b2c22b-ae6f-4a55-9da5-c67fd8786418',
 'ec7a88df-f0f3-463f-bcc3-efc054ad13b3',
 'abbe0dd9-24b5-4787-873a-59bb954d8876',
 'e88aac11-7d3d-44bd-8901-f3529f9da660',
 '91b7a49a-268e-4b45-9d8e-fd1f1d26daf5',
 'b146ef8d-3871-4a75-927b-337e839dc848',
 '3e58a0d6-8402-4e38-996c-fe3cae478f2f',
 '41f663a5-4181-4dc7-96df-0f1ffeb37c0d',
 'ba994e77-9a77-47d8-a8e7-f726b48e512c',
 '17d31542-8b0b-4708-9c9b-19d0060b5139',
 '4d34309c-5931-40a6-9c3d-e9deac1951ac',
 'b6cde8e1-1075-43dd-9ee2-ea520384a903',
 'b2fa9df7-40a4-43f8-a1ad-a48abbbe4428',
 '41655982-f190-4481-85c6-1a68d724215b',
 '79bd0a14-8d2a-4889-ae4a-346ebe674a13',
 'a5ea8402-a80d-4c77-918b-4d4201dcee7c',
 '68822fe7-17ed-4a1f-9dc8-ca462b7253e4',
 '716a2d58-eeac-47d5-907d-8ae211d8bc31',
 '43b19d26-7642-4a33-bab6-812e78f8a7a8',
 'd9144acd-f275-4bef-9bec-bf9f3bffd716',
 'e62e4de4-be00-459d-a281-ac8c938dba82',
 'bcba5e72-f0d9-43a9-8b2a-65a306134820',
 '1b949dfc-ef45-4f68-a850-31476c0c30e4',
 'e2064369-809b-4b58-8948-346ca7f2249b',
 '0bb9d151-3667-

## Make a retriever

In [9]:
retriever = vector_store.as_retriever(search_kwargs={"k": 2})

context = retriever.invoke("which company does sheryl Baxter work for?")

for i, context in enumerate(context):
    print(f"Doc : {i+1}")
    print(f"Context : {context}")

Doc : 1
Context : page_content='Index: 1
Customer Id: DD37Cf93aecA6Dc
First Name: Sheryl
Last Name: Baxter
Company: Rasmussen Group
City: East Leonard
Country: Chile
Phone 1: 229.077.5154
Phone 2: 397.884.0519x718
Email: zunigavanessa@smith.info
Subscription Date: 2020-08-24
Website: http://www.stephenson.com/' metadata={'source': '../data/customers-100.csv', 'row': 0}
Doc : 2
Context : page_content='Index: 9
Customer Id: C2dE4dEEc489ae0
First Name: Sheryl
Last Name: Meyers
Company: Browning-Simon
City: Robersonstad
Country: Cyprus
Phone 1: 854-138-4911x5772
Phone 2: +1-448-910-2276x729
Email: mariokhan@ryan-pope.org
Subscription Date: 2020-01-13
Website: https://www.bullock.net/' metadata={'source': '../data/customers-100.csv', 'row': 8}


In [10]:
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate

prompt = PromptTemplate.from_template(
    "You are a helpful assistant. That looks at what user asked in query : {query}, " \
    "and tries to answer this query using context given : {context}. If you didn't get answer in context just let it go."
)

chain = prompt | llm 

In [13]:
# function to get context and stictch it 
def get_context(retriever, query):
    retrieved_docs = retriever.invoke(query)
    context = ""
    for i, doc in enumerate(retrieved_docs):
        context += doc.page_content
        context += "\n\n"

    return context

In [16]:
import time

if __name__ == "__main__":
    query = input("Write down your query : ")
    print(f"Query : {query}")
    # sample query : which company does sheryl Baxter work for?
    start = time.time()
    context = get_context(retriever, query)
    mid = time.time()
    response = chain.invoke({'query' : query, 'context' : context})
    end = time.time()
    print(f"Context retrieval time : {mid-start :.2f} sec")
    print(f"Response generation time : {end - mid :.2f} sec")
    print(f"Total time : {end - start :.2f} sec")
    print(f"Response : {response.content}")
    print(f"-"*89)    

Query : which company does sheryl Baxter work for?
Context retrieval time : 0.30 sec
Response generation time : 10.30 sec
Total time : 10.60 sec
Response : Based on the provided context, I found that Sheryl Baxter works for Rasmussen Group.
-----------------------------------------------------------------------------------------
